In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as cma
import seaborn as sns
import time
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold as kf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve

In [46]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [154]:
class LogisticRegression:

    def __init__(self, learning_rate=0.001, n_iters=1000, alpha=1):
        self.lr = learning_rate
        self.alpha = alpha
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
        self.cost_history= [ ]
        self.w_list = [ ]

    def fit(self, X, y):
        
        n_samples, n_features = X.shape

        # initializing the parameters:
        self.weights = np.zeros(n_features)
        self.bias = 0
        eps=10**-10
        
        # gradient descent
        for _ in range(self.n_iters):
            
            # approximate y with linear combination of weights and x, plus bias
            linear_model = np.dot(X, self.weights) + self.bias
            
            # apply sigmoid function
            y_predicted = self._sigmoid(linear_model)
            
            # compute gradients
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)
            
            # update parameters:
            w_prev = self.weights
            w0_prev = self.bias
            self.weights = self.weights*(1 - self.alpha * self.lr/n_samples) - self.lr * dw
            self.bias -= self.lr * db
            
            #appending cost and w values
            
            cost = -(np.sum( y * np.log(y_predicted) + (1-y) * np.log(1 - y_predicted)) +  self.alpha*np.sum(np.square(self.weights)))
            
            w=list(self.weights)
            w.append(self.bias)
            self.w_list.append(w)
            self.cost_history.append(cost)
            
            if np.sum(np.square(w_prev-self.bias))<eps:
                break
            
    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_cls)
    
    def predict_prob(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        return y_predicted

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def score(self,x_test,y_test):
        y_p = self.predict(x_test)
        correct = 0
        for i in range(len(y_p)):
            if y_p[i]==y_test[i]:
                correct+=1
        return correct/len(y_p)
    
    def error(self,x_test,y_test):
        y_p = self.predict(x_test)
        return np.sum(np.square(y_p-y_test))

In [133]:
model = LogisticRegression(learning_rate=0.01,n_iters=200)